
---

#### *Introduction*

In this notebook, we will extend our previous work by training a convolutional neural network (CNN) to classify coins from three mints:  
`al-Mansuriyah`, `Misr`, and `al-Mahdiyah`.  
We aim to analyze the model's performance and conduct various experiments to understand the importance of different coin regions in classification.

---



In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model



In [24]:

data_dir = 'Organized_images'
classes = [ 'Misr','al-Mansuriyah', 'al-Mahdiyah']

In [25]:
# Function to load and preprocess images
def load_and_preprocess_images(data_dir, classes):
    images = []
    labels = []
    for class_label in classes:
        class_dir = os.path.join(data_dir, class_label)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array) # VGG16 preprocessing includes normalization
            images.append(img_array)
            labels.append(class_label)
    return np.array(images), np.array(labels)


In [26]:
# Load and preprocess the images
images, labels = load_and_preprocess_images(data_dir, classes)

# Encode labels as integers
label_to_index = {label: idx for idx, label in enumerate(classes)}
index_to_label = {idx: label for label, idx in label_to_index.items()}
labels = np.array([label_to_index[label] for label in labels])

In [27]:
# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val, shuffle=True)

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=len(classes))
y_val = to_categorical(y_val, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 306
Validation set size: 77
Test set size: 96


In [28]:
# Load pre-trained VGG16 model + higher level layers
def create_model(num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Add custom top layers for transfer learning
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Combine base model and new top layers
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model (not trainable)
    for layer in base_model.layers:
        layer.trainable = False

    return model

In [29]:
# Create and compile the model
num_classes = len(classes)
model = create_model(num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Save the trained model
model.save('three_class_model.h5')

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.4159 - loss: 144.5408 - val_accuracy: 0.4935 - val_loss: 57.0434
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.6179 - loss: 41.6920 - val_accuracy: 0.6234 - val_loss: 14.4348
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.5913 - loss: 26.5641 - val_accuracy: 0.7273 - val_loss: 13.4603
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.7324 - loss: 14.6720 - val_accuracy: 0.7273 - val_loss: 10.2937
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.8103 - loss: 10.2503 - val_accuracy: 0.8312 - val_loss: 6.5183
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.8575 - loss: 5.2736 - val_accuracy: 0.8442 - val_loss: 6.7812
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.8652 - loss: 4.0666 - val_accuracy: 0.8312 - val_loss: 6.0921
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.8236 - loss: 7.2868 - val_accuracy: 0.7532 -

In [30]:
# Evaluate model performance on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {accuracy:.4f}")

Test accuracy: 0.8438


In [ ]:
# Load the trained model
model = load_model('three_class_model.h5')

In [12]:
# Rotate on all the images in Organized_images/al-Mahdiyah and visualize the results with gradcam


# Load and preprocess the images
# images = []
# for img_name in os.listdir('Organized_images/al-Mahdiyah'):
#     img_path = os.path.join('Organized_images/al-Mahdiyah', img_name)
#     img = load_img(img_path, target_size=(224, 224))
#     img_array = img_to_array(img)
#     img_array = preprocess_input(img_array)
#     images.append(img_array)
# images = np.array(images)

# Predict the class of each image
predictions = model.predict(X_test)
predicted_labels = predictions.argmax(axis=1)
print(predicted_labels)

3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step
[1 0 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0
 1 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0]


In [17]:
y_test_al_mahdiyah_index = np.where(y_test.argmax(axis=1) == 2)[0]


In [18]:
y_test_al_mahdiyah_index

array([ 7, 30, 31, 34, 44, 66, 75], dtype=int64)

In [22]:
print(np.argmax(predictions[y_test_al_mahdiyah_index], axis=1))

[0 0 0 1 0 0 0]
